# SnapBack the Smart Hat

Snap back your happy moments.

<img src="img/githubSnapBackCover.jpg"/>

### Team Members
##### **Billy Brickner**
##### **Nhan Tran**
##### **Huan Wang**
##### **Tyler Quast**



## How it works

Webcam-equipped cap that captures happy moments by recognizing smiling faces and using that recognition to trigger the capture of a short video clip, which is then wirelessly uploaded to the cloud for later viewing.

<img src="img/SnapBack.jpg">

# Hardware Setup

Our hardware has been designed to be portable to accomodate for the active lifestyle of our memory makers. To make this possible we made some hardware modifications to accomadate for a chordless device.


## Battery Pack

For power on the go, a power block was repurposed to connect the board to a 9V power source. In this case the power source is three 9V batteries connected in parralel with electrical tape.


<img src="img/BatteryPack.jpg">

## Peripherals

The peripherals that were used were from left to right:
**Arduino Shield** - For powering the periferals
**USB Webcam** - For capturing memories
**USB Wifi** - A Wifi Adapter for uploading memories to the cloud.
**USB Splitter** - A Device for allowing multiple inputs into the board


<img src="img/Periferals.jpg">

## Arduino Shield

The ardunio shield was modified to provide power to the peripherals by soldering the usb splitter power wire to the 5V pin on the board.  

<img src ='img/ArduinoShield.jpg' />

## Protective Case

This protective case hold the components together in a compact and discrete manner while you walk around with the snapback.

<img src="img/ProtectiveCase.jpg" />

#### Everything is in the case! Completely cordless! You can walk around and snapback will capture your happy moments!

<img src="img/enclosedBox.jpg" />


# Website Server Set up

## 1. Install all the dependencies
 - pip install dropbox
 - pip install flask

## 2. cd into the Webserver folder
expert the path for flask

 export FLASK_APP=main.py


## 3. Run:
 - flask run

to start the server. Go to localhost:5000 to view the site.

<img src="img/website_demo.jpg" />

# Vision Code

In terminal, run python3.6 smile_detector.py

In [ ]:
#!/usr/bin/python3.6

from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import numpy as np
from pynq import Xlnk
import time
import os
import cv2

xlnk = Xlnk()
xlnk.xlnk_reset()

base = BaseOverlay("base.bit")

face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_smile.xml')

frame_w = 640 
frame_h = 480

Mode = VideoMode(frame_w, frame_h, 24) 
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_BGR)
hdmi_out.start()

cap = cv2.VideoCapture(0)
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Capture device is open? " + str(cap.isOpened()))

#Global Vars
start_recording = False
trigger_time = 0 #Hold timestamp of the last detection

num_smiles = 0

fourcc = cv2.VideoWriter_fourcc(*'H264')  
writer = None
# Facial and Smile recognition:


try:
    started = time.time()
    while base.buttons[1].read()==0:
            ret, frame_vga = cap.read()
            if (ret):
                """
                outframe = hdmi_out.newframe()
                outframe[:] = frame_vga
                hdmi_out.writeframe(outframe)
                """
                
                np_frame = frame_vga
                
                instant = time.time() # get timestamp of the frame

                      
                gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
                

                faces = face_cascade.detectMultiScale(gray, 1.3, 5)

                for (x,y,w,h) in faces:
                    cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
                    roi_gray = gray[y:y+h, x:x+w]
                    roi_color = np_frame[y:y+h, x:x+w]
                    """
                    eyes = eye_cascade.detectMultiScale(roi_gray)
                    for (ex,ey,ew,eh) in eyes:
                        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    """
                    smiles = smile_cascade.detectMultiScale(
                        roi_gray,
                        scaleFactor= 1.7,
                        minNeighbors=22,
                        minSize=(25, 25),
                        flags=cv2.CASCADE_SCALE_IMAGE
                    )
                    for (sx,sy,sw,sh) in smiles:
                        print("Found", len(smiles), "smile(s).")
                        cv2.rectangle(roi_color, (sx,sy), (sx+sw, sy+sh), (255, 0,0), 1)
                        #display number of smile on frame:
                        text_font = cv2.FONT_HERSHEY_SIMPLEX
                        bottomLeftCornerOfText = (0,300)
                        fontScale = 1
                        fontColor = (255,255,255)
                        lineType = 2        
                        cv2.putText(np_frame,"Found {} smile(s)!".format(len(smiles)), 
                            bottomLeftCornerOfText, 
                            text_font, 
                            fontScale,
                            fontColor,
                            lineType)
                    if len(smiles) > 0:
                        if writer is None:
                            writer =cv2.VideoWriter('capture_moments/'+time.strftime("%Y%m%d-%H%M%S") + '.mp4',fourcc, 2.5, (frame_w,frame_h))

                        num_smiles = len(smiles)
                        trigger_time = instant
                        start_recording = True
                    if start_recording and instant <= trigger_time + 30:
                        writer.write(frame_vga)
                        print("writing because ", instant, trigger_time + 10)
                    else:
                        print("STOP RECORDING")
                        start_recording = False
                        if writer is not None:
                            writer.release()
                            os.system("python3.6 dropbox_support.py")
                            writer = None
                        
                        #capture_video('captured_moments', 5)
                        
                        # capture smiling images
                        #cv2.imwrite('capture_moments/' + time.strftime("%Y%m%d-%H%M%S") + '.jpg', frame_vga)
                        

                # Output OpenCV results via HDMI
                outframe = hdmi_out.newframe()
                outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]

                #outframe = hdmi_out.newframe()
                #outframe[:] = frame
                
                hdmi_out.writeframe(outframe)
                
            else:
                raise RuntimeError("Error while reading from camera")
finally:
    print("Cancel")
    hdmi_out.stop()
    cap.release()
    if writer is not None:
        writer.release()
        os.system("python3.6 dropbox_support.py")
    cv2.destroyAllWindows()